In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.request


In [2]:
url = "https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2380057.m570.l1313&_nkw=goku&_sacat=0"
request = requests.get(url)
print(request)
soup = BeautifulSoup(request.text, 'html.parser')
type(soup)

<Response [200]>


bs4.BeautifulSoup

In [3]:
results = soup.findAll("div", {'class': "s-item__wrapper clearfix"})
items = []
for result in results[1:]:
    productInfo = {}
    
    productInfo["title"] = result.find('h3', {'class': 's-item__title'}).text.replace("New Listing", '')

    try:
        productInfo["status"] = result.find("div", {'class': 's-item__subtitle'}).span.text
    except:
        print("OHHHHHHHHHHHHHHH")


    productInfo["price"] = float(re.search("\d+\.\d+" ,result.find(
        "span", {'class': 's-item__price'}).text).group().replace("$", ""))
   #got the minimum because it was a range
    try:
        productInfo["watchers"] = int(re.search(r"\d+", result.find(
            "span", {'class': 'BOLD NEGATIVE'}).text).group())
    except:
        pass
    try:
        productInfo["country"] = result.find("span", {'class': 's-item__location s-item__itemLocation'}).text.replace(
        "from ", "")
    except:
        pass
    items.append(productInfo)

In [6]:
itemsDF = pd.DataFrame(items)
try:
    itemsDF["watchers"].fillna(0, inplace = True)
except:
    pass
itemsDF.to_csv("ebay.csv", index = False)
print("Saved")


Saved
